In [23]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import voronoi_diagram
from shapely.affinity import scale
import json
from shapely.geometry import MultiPoint
import colorsys

In [24]:
file_path = 'dtc_summary.csv'

# Load CSV and prepare GeoDataFrame
df = pd.read_csv(file_path)
df['addrLine1'] = df['addrLine1'].fillna('')
df['addrLine2'] = df['addrLine2'].fillna('')
df['addrLine3'] = df['addrLine3'].fillna('')
df['addrLine4'] = df['addrLine4'].fillna('')
df['addrLine5'] = df['addrLine5'].fillna('')

#get pass rate percentiles 10% and 90%
low_pass_rate = df['pass'].quantile(0.05)
high_pass_rate = df['pass'].quantile(0.9)

In [25]:
#print all percentiles of dailyTestCount
print(df['dailyTestCount'].quantile([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))

0.1     0.59213
0.2     2.85298
0.3     7.59810
0.4    11.23258
0.5    14.29535
0.6    16.60838
0.7    20.66933
0.8    24.40840
0.9    29.19256
Name: dailyTestCount, dtype: float64


In [26]:

def pass_rate_to_color(pass_rate):
    pass_rate = max(low_pass_rate, min(high_pass_rate, pass_rate))    
    perc_inside = (pass_rate - low_pass_rate) / (high_pass_rate - low_pass_rate)
    hue = perc_inside * 0.67 #0 = red, 0.67 = blue
    r, g, b = colorsys.hls_to_rgb(hue, 0.5, 1.0)
    return f'#{int(r * 255):02x}{int(g * 255):02x}{int(b * 255):02x}'

def daily_test_count_to_radius(daily_test_count):
    # MIN(CEIL(2 + daily_test_count / 8), 10)
    return min(2 + daily_test_count / 8, 10)

def daily_test_count_to_size_string(daily_test_count):
    if daily_test_count < 8:
        return 'small'
    elif daily_test_count < 16:
        return 'medium'
    else:
        return 'large'

In [27]:
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")
gdf = gdf.to_crs(epsg=32630)  # Convert to metric CRS for Voronoi calculation
points = MultiPoint(gdf.geometry.tolist())
voronoi_cells = voronoi_diagram(points)
features = []
for i, cell in enumerate(voronoi_cells.geoms):
    row = gdf[cell.contains(gdf.geometry)].iloc[0]
    center_point = row.geometry
    buffer_polygon = center_point.buffer(1609.34 * 5)  # miles to meters
    clipped_polygon = cell.intersection(buffer_polygon)
    properties = row.drop('geometry').to_dict()
    properties['fill'] = pass_rate_to_color(properties['pass'])
    properties['fill-opacity'] = 0.7
    clipped_polygon = gpd.GeoSeries([clipped_polygon], crs="EPSG:32630").to_crs(epsg=4326).iloc[0]
    features.append({
        "type": "Feature",
        "geometry": json.loads(gpd.GeoSeries([clipped_polygon]).to_json())['features'][0]['geometry'],
        "properties": properties
    })

# Construct GeoJSON output
geojson_output = {
    "type": "FeatureCollection",
    "features": features
}

# Write GeoJSON to file
with open("dtcs.geojson", "w") as geojson_file:
    json.dump(geojson_output, geojson_file)


In [28]:
# dot = {
#     "type": "Feature",
#     "geometry": {
#         "type": "Point",
#         "coordinates": [row['longitude'], row['latitude']]
#     },
#     "properties": {
#         "marker-color": pass_rate_to_color(properties['pass']),
#         "radius": daily_test_count_to_radius(properties['dailyTestCount']),
#         "marker-size": daily_test_count_to_size_string(properties['dailyTestCount']),
#     }
# }
# features.append(dot)